# 01 Data Cleaning and Feature Engineering
## Online Retail Analysis Project

This notebook covers:
1. Data Loading
2. Exploratory Data Analysis (EDA)
3. Data Cleaning
4. Feature Engineering
5. Data Export for Modeling

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 2. Load Data

In [ ]:
# Load raw data
df = pd.read_csv('../data/raw_data.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Basic information
print("Dataset Info:")
print(df.info())
print("\n" + "="*50)
print("\nBasic Statistics:")
print(df.describe())

In [ ]:
# Check for missing values
print("Missing Values:")
missing = df.isnull().sum()
missing_percent = 100 * df.isnull().sum() / len(df)
missing_table = pd.concat([missing, missing_percent], axis=1, keys=['Total', 'Percent'])
print(missing_table[missing_table['Total'] > 0])

In [ ]:
# Check for duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

In [ ]:
# Unique values in key columns
print("Unique Counts:")
print(f"Unique Invoices: {df['InvoiceNo'].nunique()}")
print(f"Unique Products: {df['StockCode'].nunique()}")
print(f"Unique Customers: {df['CustomerID'].nunique()}")
print(f"Unique Countries: {df['Country'].nunique()}")

## 4. Data Cleaning

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Convert InvoiceDate to datetime
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

# Remove rows with missing CustomerID (if any)
df_clean = df_clean.dropna(subset=['CustomerID'])

# Convert CustomerID to integer
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)

# Remove cancelled transactions (invoices starting with 'C')
df_clean = df_clean[~df_clean['InvoiceNo'].astype(str).str.startswith('C')]

# Remove rows with negative or zero quantities
df_clean = df_clean[df_clean['Quantity'] > 0]

# Remove rows with negative or zero prices
df_clean = df_clean[df_clean['UnitPrice'] > 0]

print(f"Original dataset shape: {df.shape}")
print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Rows removed: {df.shape[0] - df_clean.shape[0]}")

## 5. Feature Engineering

In [ ]:
# Calculate total price for each transaction
df_clean['TotalPrice'] = df_clean['Quantity'] * df_clean['UnitPrice']

# Extract date components
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['Day'] = df_clean['InvoiceDate'].dt.day
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.dayofweek
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour

# Create day name
df_clean['DayName'] = df_clean['InvoiceDate'].dt.day_name()

print("New features created:")
print(df_clean[['InvoiceDate', 'TotalPrice', 'Year', 'Month', 'Day', 'DayOfWeek', 'Hour', 'DayName']].head())

## 6. Customer-Level Features (RFM Analysis Preparation)

In [ ]:
# Calculate snapshot date (1 day after the last transaction)
snapshot_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)

# Create RFM features for each customer
rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency
    'TotalPrice': 'sum'  # Monetary
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm = rfm.reset_index()

print("RFM Analysis:")
print(rfm.head(10))
print(f"\nRFM Statistics:")
print(rfm.describe())

## 7. Product-Level Features

In [ ]:
# Product popularity
product_stats = df_clean.groupby('StockCode').agg({
    'Quantity': 'sum',
    'TotalPrice': 'sum',
    'InvoiceNo': 'nunique',
    'Description': 'first'
}).reset_index()

product_stats.columns = ['StockCode', 'TotalQuantitySold', 'TotalRevenue', 'NumberOfOrders', 'Description']
product_stats = product_stats.sort_values('TotalRevenue', ascending=False)

print("Top 10 Products by Revenue:")
print(product_stats.head(10))

## 8. Visualizations

In [ ]:
# Sales over time
daily_sales = df_clean.groupby(df_clean['InvoiceDate'].dt.date)['TotalPrice'].sum().reset_index()
daily_sales.columns = ['Date', 'Revenue']

plt.figure(figsize=(14, 6))
plt.plot(daily_sales['Date'], daily_sales['Revenue'], linewidth=2)
plt.title('Daily Revenue Trend', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Revenue (£)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Top 10 countries by revenue
country_revenue = df_clean.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
country_revenue.plot(kind='barh', color='skyblue')
plt.title('Top 10 Countries by Revenue', fontsize=16, fontweight='bold')
plt.xlabel('Revenue (£)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Sales by hour of day
hourly_sales = df_clean.groupby('Hour')['TotalPrice'].sum()

plt.figure(figsize=(12, 6))
hourly_sales.plot(kind='bar', color='coral')
plt.title('Sales Distribution by Hour of Day', fontsize=16, fontweight='bold')
plt.xlabel('Hour', fontsize=12)
plt.ylabel('Revenue (£)', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Sales by day of week
day_sales = df_clean.groupby('DayName')['TotalPrice'].sum().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

plt.figure(figsize=(12, 6))
day_sales.plot(kind='bar', color='lightgreen')
plt.title('Sales Distribution by Day of Week', fontsize=16, fontweight='bold')
plt.xlabel('Day', fontsize=12)
plt.ylabel('Revenue (£)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. Save Processed Data

In [ ]:
# Save cleaned and processed data
df_clean.to_csv('../data/processed_data.csv', index=False)
print(f"Processed data saved to '../data/processed_data.csv'")
print(f"Shape: {df_clean.shape}")
print(f"\nColumns: {list(df_clean.columns)}")

In [ ]:
# Save RFM data for customer segmentation
rfm.to_csv('../data/rfm_data.csv', index=False)
print("RFM data saved to '../data/rfm_data.csv'")

In [ ]:
# Save product statistics
product_stats.to_csv('../data/product_stats.csv', index=False)
print("Product statistics saved to '../data/product_stats.csv'")

## Summary

In this notebook, we:
1. Loaded the raw e-commerce dataset
2. Performed exploratory data analysis to understand the data
3. Cleaned the data by removing invalid transactions and missing values
4. Engineered features including:
   - Transaction total price
   - Date/time components
   - RFM (Recency, Frequency, Monetary) metrics for customers
   - Product-level statistics
5. Created visualizations to understand sales patterns
6. Saved the processed data for modeling and analysis

Next steps: Move to the modeling and analysis notebook for customer segmentation and product recommendations.